In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))
import zipfile
import sys
import time

from keras import backend, models, layers, initializers, regularizers, constraints, optimizers
from keras import callbacks as kc
from keras import optimizers as ko

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import log_loss
import time

['sample_submission_stage_1.csv', 'sample_submission_stage_2.csv', 'test_stage_1.tsv', 'trees']


Using TensorFlow backend.


# Generate BERT embeddings for test set

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))
import zipfile
import sys
import time

import modeling
import extract_features
import tokenization
import tensorflow as tf

def compute_offset_no_spaces(text, offset):
    count = 0
    for pos in range(offset):
        if text[pos] != " ": count +=1
    return count

def count_chars_no_special(text):
    count = 0
    special_char_list = ["#"]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: count +=1
    return count

def count_length_no_special(text):
    count = 0
    special_char_list = ["#", " "]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: count +=1
    return count

def run_bert(data, output, os_command, embedding_size=1024, layer=-1):
    '''
    Runs a forward propagation of BERT on input text, extracting contextual word embeddings
    Input: data, a pandas DataFrame containing the information in one of the GAP files

    Output: emb, a pandas DataFrame containing contextual embeddings for the words A, B and Pronoun. Each embedding is a numpy array of shape (768)
    columns: "emb_A": the embedding for word A
             "emb_B": the embedding for word B
             "emb_P": the embedding for the pronoun
             "label": the answer to the coreference problem: "A", "B" or "NEITHER"
    '''
    # From the current file, take the text only, and write it in a file which will be passed to BERT
    text = data["Text"]
    text.to_csv("input.txt", index = False, header = False)

    # The script extract_features.py runs forward propagation through BERT, and writes the output in the file output.jsonl
    # I'm lazy, so I'm only saving the output of the last layer. Feel free to change --layers = -1 to save the output of other layers.

    res = os.system(os_command)
    bert_output = pd.read_json("output.jsonl", lines = True)

    os.system("rm output.jsonl")
    os.system("rm input.txt")

    index = data.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index = index, columns = columns)
    emb.index.name = "ID"

    for i in range(len(data)): # For each line in the data file
        # get the words A, B, Pronoun. Convert them to lower case, since we're using the uncased version of BERT
        P = data.loc[i,"Pronoun"].lower()
        A = data.loc[i,"A"].lower()
        B = data.loc[i,"B"].lower()

        # For each word, find the offset not counting spaces. This is necessary for comparison with the output of BERT
        P_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"Pronoun-offset"])
        A_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"A-offset"])
        B_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"B-offset"])
        # Figure out the length of A, B, not counting spaces or special characters
        A_length = count_length_no_special(A)
        B_length = count_length_no_special(B)

        # Initialize embeddings with zeros
        emb_A = np.zeros(embedding_size)
        emb_B = np.zeros(embedding_size)
        emb_P = np.zeros(embedding_size)

        # Initialize counts
        count_chars = 0
        cnt_A, cnt_B, cnt_P = 0, 0, 0

        features = pd.DataFrame(bert_output.loc[i,"features"]) # Get the BERT embeddings for the current line in the data file
        for j in range(2,len(features)):  # Iterate over the BERT tokens for the current line; we skip over the first 2 tokens, which don't correspond to words
            token = features.loc[j,"token"]

            # See if the character count until the current token matches the offset of any of the 3 target words
            if count_chars  == P_offset: 
                # print(token)
                emb_P += np.array(features.loc[j,"layers"][0]['values'])
                cnt_P += 1
            if count_chars in range(A_offset, A_offset + A_length): 
                # print(token)
                emb_A += np.array(features.loc[j,"layers"][0]['values'])
                cnt_A +=1
            if count_chars in range(B_offset, B_offset + B_length): 
                # print(token)
                emb_B += np.array(features.loc[j,"layers"][0]['values'])
                cnt_B +=1								
            # Update the character count
            count_chars += count_length_no_special(token)
        # Taking the average between tokens in the span of A or B, so divide the current value by the count	
        emb_A /= cnt_A
        emb_B /= cnt_B

        # Work out the label of the current piece of text
        label = "Neither"
        if (data.loc[i,"A-coref"] == True):
            label = "A"
        if (data.loc[i,"B-coref"] == True):
            label = "B"

        # Put everything together in emb
        emb.iloc[i] = [emb_A, emb_B, emb_P, label]
    return emb

['sample_submission_stage_1.csv', 'sample_submission_stage_2.csv', 'test_stage_1.tsv', 'trees']


In [ ]:
stage_2_dataset_name = 'test_stage_2.tsv' # remember to modify this from 'test_stage_1.tsv' -> 'test_stage_2.tsv', or the code cannot run

In [ ]:
test_data = pd.read_csv(stage_2_dataset_name, sep='\t')
test_data['A-coref'] = False
test_data['B-coref'] = False
test_data['ID'] = 'development' + test_data['ID']
test_data[['ID', 'Text', 'Pronoun', 'Pronoun-offset', 'A', 'A-offset', 'A-coref', 
          'B', 'B-offset', 'B-coref', 'URL']].to_csv("fixed_test_stage_2.tsv", index=False, sep='\t')

## Bert uncased base

In [3]:
tag = 'bert-base-uncased-seq512-'
embedding_size = 768
layer = 8
tag = tag + str(layer)
os_command = "python extract_features.py \
      --input_file=input.txt \
      --output_file=output.jsonl \
      --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
      --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
      --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
      --layers={} \
      --max_seq_length=512 \
      --batch_size=1".format(layer)

test_data = pd.read_csv("fixed_test_stage_2.tsv", sep = '\t')
test_emb = run_bert(test_data, embedding_size=embedding_size, os_command=os_command,
                    output="{}contextual_embeddings_gap_test.json".format(tag), layer=layer)
test_emb.to_json("vector/bert_base/{}contextual_embeddings_gap_second_stage_test.json".format(tag), orient = 'columns')

C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in true_divide
C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in true_divide


## Bert cased base

In [ ]:
tag = 'bert-base-cased-seq512-'
embedding_size = 768
layer = 8
tag = tag + str(layer)
os_command = "python extract_features.py \
      --input_file=input.txt \
      --output_file=output.jsonl \
      --vocab_file=cased_L-12_H-768_A-12/vocab.txt \
      --bert_config_file=cased_L-12_H-768_A-12/bert_config.json \
      --init_checkpoint=cased_L-12_H-768_A-12/bert_model.ckpt \
      --layers={} \
      --max_seq_length=512 \
      --batch_size=1".format(layer)

test_data = pd.read_csv("fixed_test_stage_2.tsv", sep = '\t')
test_emb = run_bert(test_data, embedding_size=embedding_size, os_command=os_command,
                    output="{}contextual_embeddings_gap_test.json".format(tag), layer=layer)
test_emb.to_json("vector/bert_base_cased/{}contextual_embeddings_gap_second_stage_test.json".format(tag), orient = 'columns')

C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in true_divide
C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in true_divide


## Bert uncased large

In [ ]:
tag = 'bert-large-uncased-seq300-'
embedding_size = 1024
layer = 19
tag = tag + str(layer)
os_command = "python extract_features.py \
      --input_file=input.txt \
      --output_file=output.jsonl \
      --vocab_file=uncased_L-24_H-1024_A-16/vocab.txt \
      --bert_config_file=uncased_L-24_H-1024_A-16/bert_config.json \
      --init_checkpoint=uncased_L-24_H-1024_A-16/bert_model.ckpt \
      --layers={} \
      --max_seq_length=300 \
      --batch_size=1".format(layer)

test_data = pd.read_csv("fixed_test_stage_2.tsv", sep = '\t')
test_emb = run_bert(test_data, embedding_size=embedding_size, os_command=os_command,
                    output="{}contextual_embeddings_gap_test.json".format(tag), layer=layer)
test_emb.to_json("vector/bert_big/{}contextual_embeddings_gap_second_stage_test.json".format(tag), orient = 'columns')

C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in true_divide
C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in true_divide


## Bert cased large

In [ ]:
tag = 'bert-large-cased-seq300-'
embedding_size = 1024
layer = 18
tag = tag + str(layer)
os_command = "python extract_features.py \
      --input_file=input.txt \
      --output_file=output.jsonl \
      --vocab_file=cased_L-24_H-1024_A-16/vocab.txt \
      --bert_config_file=cased_L-24_H-1024_A-16/bert_config.json \
      --init_checkpoint=cased_L-24_H-1024_A-16/bert_model.ckpt \
      --layers={} \
      --max_seq_length=300 \
      --batch_size=1".format(layer)

test_data = pd.read_csv("fixed_test_stage_2.tsv", sep = '\t')
test_emb = run_bert(test_data, embedding_size=embedding_size, os_command=os_command,
                    output="{}contextual_embeddings_gap_test.json".format(tag), layer=layer)
test_emb.to_json("vector/bert_big_cased/{}contextual_embeddings_gap_second_stage_test.json".format(tag), orient = 'columns')

In [3]:
def parse_json(embeddings, embedding_size):
    '''
    Parses the embeddigns given by BERT, and suitably formats them to be passed to the MLP model

    Input: embeddings, a DataFrame containing contextual embeddings from BERT, as well as the labels for the classification problem
    columns: "emb_A": contextual embedding for the word A
             "emb_B": contextual embedding for the word B
             "emb_P": contextual embedding for the pronoun
             "label": the answer to the coreference problem: "A", "B" or "NEITHER"

    Output: X, a numpy array containing, for each line in the GAP file, the concatenation of the embeddings of the target words
            Y, a numpy array containing, for each line in the GAP file, the one-hot encoded answer to the coreference problem
    '''
    embeddings.sort_index(inplace = True) # Sorting the DataFrame, because reading from the json file messed with the order
    X = np.zeros((len(embeddings),3* embedding_size))
    Y = np.zeros((len(embeddings), 3))

    # Concatenate features
    for i in range(len(embeddings)):
        A = np.array(embeddings.loc[i,"emb_A"])
        B = np.array(embeddings.loc[i,"emb_B"])
        P = np.array(embeddings.loc[i,"emb_P"])
        X[i] = np.concatenate((A,B,P))

    # One-hot encoding for labels
    for i in range(len(embeddings)):
        label = embeddings.loc[i,"label"]
        if label == "A":
            Y[i,0] = 1
        elif label == "B":
            Y[i,1] = 1
        else:
            Y[i,2] = 1

    return X, Y

## Predictons for stage 1.2

In [10]:
dev_filename = "contextual_embeddings_gap_second_stage_test.json"
val_filename = "contextual_embeddings_gap_validation.json"
test_filename = "contextual_embeddings_gap_test.json"

In [11]:
dense_layer_sizes = [37]
dropout_rate = 0.6
learning_rate = 0.001
n_fold = 5
batch_size = 32
epochs = 1000
patience = 100
# n_test = 100
lambd = 0.1 # L2 regularization

def build_mlp_model(input_shape):
	X_input = layers.Input(input_shape)
	X = layers.Dropout(dropout_rate, seed = 7)(X_input)
	# First dense layer
	X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(X)
	X = layers.BatchNormalization(name = 'bn0')(X)
	X = layers.Activation('relu')(X)
	X = layers.Dropout(dropout_rate, seed = 7)(X)

	# Output layer
	X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
	X = layers.Activation('softmax')(X)

	# Create model
	model = models.Model(input = X_input, output = X, name = "classif_model")
	return model

In [14]:
def make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_tag,
                dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
                oof_folder='oof/', pred_folder='test_outputs/'):

    tag = tag + str(layer) # follow the original naming style
    dev_filename = tag + dev_filename
    val_filename = tag + val_filename
    test_filename = tag + test_filename
    
    development = pd.read_json(os.path.join(dev_folder_path, dev_filename))
    X_development, Y_development = parse_json(development, embedding_size)

    # We want predictions for all development rows. So instead of removing rows, make them 0
    remove_development = [row for row in range(len(X_development)) if np.sum(np.isnan(X_development[row].reshape(-1)))]
    X_development[remove_development] = np.zeros((3 * embedding_size))

    prediction = np.zeros((len(X_development), 3)) # testing predictions

    for fold_n in range(n_fold):
        # split training and validation data
        print('Make predictions for', fold_n, 'started at', time.ctime())
        classif_model = build_mlp_model([X_development.shape[-1]])
        classif_model.load_weights(os.path.join(checkpoint_path, model_tag + tag + str(fold_n) + '.pt'))

        pred = classif_model.predict(x=X_development, verbose=0)
        prediction += pred
    
    prediction /= n_fold
    # prediction[remove_development] = [0.4, 0.4, 0.2] This is useless for prediction stage

    submission = pd.read_csv("data/sample_submission_stage_2.csv", index_col = "ID")
    submission["A"] = prediction[:,0]
    submission["B"] = prediction[:,1]
    submission["NEITHER"] = prediction[:,2]
    submission.to_csv(pred_folder + model_tag + tag + ".csv")

In [15]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-uncased-seq512-"
pred_tag = "naive-"

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 21:52:58 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:52:59 2019
Make predictions for 2 started at Fri Apr 19 21:53:00 2019
Make predictions for 3 started at Fri Apr 19 21:53:00 2019
Make predictions for 4 started at Fri Apr 19 21:53:01 2019
Make predictions for 5 started at Fri Apr 19 21:53:02 2019
Make predictions for 6 started at Fri Apr 19 21:53:03 2019


In [16]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base_cased"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-cased-seq512-"
pred_tag = "naive-"
make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 21:53:10 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:53:11 2019
Make predictions for 2 started at Fri Apr 19 21:53:12 2019
Make predictions for 3 started at Fri Apr 19 21:53:13 2019
Make predictions for 4 started at Fri Apr 19 21:53:14 2019
Make predictions for 5 started at Fri Apr 19 21:53:15 2019
Make predictions for 6 started at Fri Apr 19 21:53:16 2019


In [17]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big"
embedding_size = 1024
layer = 19
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-uncased-seq300-"
pred_tag = "naive-"
make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 21:53:21 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:53:22 2019
Make predictions for 2 started at Fri Apr 19 21:53:23 2019
Make predictions for 3 started at Fri Apr 19 21:53:24 2019
Make predictions for 4 started at Fri Apr 19 21:53:25 2019
Make predictions for 5 started at Fri Apr 19 21:53:26 2019
Make predictions for 6 started at Fri Apr 19 21:53:28 2019


In [18]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big_cased"
embedding_size = 1024
layer = 18
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-cased-seq300-"
pred_tag = "naive-"
make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 21:53:34 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:53:35 2019
Make predictions for 2 started at Fri Apr 19 21:53:36 2019
Make predictions for 3 started at Fri Apr 19 21:53:37 2019
Make predictions for 4 started at Fri Apr 19 21:53:38 2019
Make predictions for 5 started at Fri Apr 19 21:53:39 2019
Make predictions for 6 started at Fri Apr 19 21:53:40 2019


## Predictons for stage 1.3

### Model Architectures

In [19]:
def build_mlp_model_for_base_bert(input_shape, split_size):
    X_input = layers.Input(input_shape)

    # First dense layer
    A = layers.Lambda(lambda x: x[:, :split_size])(X_input)
    B = layers.Lambda(lambda x: x[:, split_size:split_size*2])(X_input)
    P = layers.Lambda(lambda x: x[:, split_size*2:])(X_input)

    head_num = 6
    res = []
    for head in range(head_num):
        query_encoder = layers.Dense(dense_layer_sizes[0], activation='selu', kernel_regularizer = regularizers.l2(1e-6))
        ans_encoder = layers.Dense(dense_layer_sizes[0], activation='selu', kernel_regularizer = regularizers.l2(1e-6))
        
        d_ratio = 0.6
        
        a, b = query_encoder(layers.Dropout(d_ratio)(A)), query_encoder(layers.Dropout(d_ratio)(B))
        p = query_encoder(layers.Dropout(d_ratio)(P))
        
        amp = layers.Multiply()([a, p])
        bmp = layers.Multiply()([b, p])
        
        asp = layers.Lambda(lambda v: v[0] - v[1])([p, a])
        bsp = layers.Lambda(lambda v: v[0] - v[1])([p, b])        
        
        ia = layers.Concatenate()([a, p, amp, asp])
        ib = layers.Concatenate()([b, p, bmp, bsp])
        nli_encoder = layers.Dense(dense_layer_sizes[0], activation='selu')
        ia, ib = nli_encoder(ia), nli_encoder(ib)
        
        out = layers.Concatenate()([ia, ib])
        res.append(out)
    
    res = layers.Add()(res)
    res = layers.Dropout(0.8)(res)
    X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(res)
    X = layers.BatchNormalization(name = 'bn0')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed = 7)(X)

    # Output layer
    X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
    X = layers.Activation('softmax')(X)

    # Create model
    model = models.Model(input = X_input, output = X, name = "classif_model")
    return model

In [20]:
def build_mlp_model_for_larget_bert(input_shape, split_size):
    X_input = layers.Input(input_shape)

    # First dense layer
    A = layers.Lambda(lambda x: x[:, :split_size])(X_input)
    B = layers.Lambda(lambda x: x[:, split_size:split_size*2])(X_input)
    P = layers.Lambda(lambda x: x[:, split_size*2:])(X_input)

    head_num = 6
    res = []
    for head in range(head_num):
        query_encoder = layers.Dense(dense_layer_sizes[0], activation='selu', kernel_regularizer = regularizers.l2(1e-6))
        ans_encoder = layers.Dense(dense_layer_sizes[0], activation='selu', kernel_regularizer = regularizers.l2(1e-6))
        
        d_ratio = 0.7
        
        a, b = query_encoder(layers.Dropout(d_ratio)(A)), query_encoder(layers.Dropout(d_ratio)(B))
        p = query_encoder(layers.Dropout(d_ratio)(P))
        
        amp = layers.Multiply()([a, p])
        bmp = layers.Multiply()([b, p])
        
        asp = layers.Lambda(lambda v: v[0] - v[1])([p, a])
        bsp = layers.Lambda(lambda v: v[0] - v[1])([p, b])        
        
        ia = layers.Concatenate()([a, p, amp, asp])
        ib = layers.Concatenate()([b, p, bmp, bsp])
        nli_encoder = layers.Dense(dense_layer_sizes[0], activation='selu')
        ia, ib = nli_encoder(ia), nli_encoder(ib)
        
        out = layers.Concatenate()([ia, ib])
        res.append(out)
    
    res = layers.Concatenate()(res)
    res = layers.Dropout(0.85)(res)
    X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(res)
    X = layers.BatchNormalization(name = 'bn0')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed = 7)(X)
    
    # Output layer
    X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
    X = layers.Activation('softmax')(X)

    # Create model
    model = models.Model(input = X_input, output = X, name = "classif_model")
    return model

'''
------------------------------
For the model bert-large-uncased-seq256-19
CV mean score: 0.3441, std: 0.0215.
[0.3307294825242308, 0.3187784074490988, 0.34923320252765083, 0.3821268940718645, 0.3397000497226646]
Test score: 0.32621788223567494
------------------------------
'''

'\n------------------------------\nFor the model bert-large-uncased-seq256-19\nCV mean score: 0.3441, std: 0.0215.\n[0.3307294825242308, 0.3187784074490988, 0.34923320252765083, 0.3821268940718645, 0.3397000497226646]\nTest score: 0.32621788223567494\n------------------------------\n'

In [21]:
import keras.backend as K

def base_nli_model(input_shape, split_size):
    X_input = layers.Input(input_shape)

    # First dense layer
    A = layers.Lambda(lambda x: x[:, :split_size])(X_input)
    B = layers.Lambda(lambda x: x[:, split_size:split_size*2])(X_input)
    P = layers.Lambda(lambda x: x[:, split_size*2:])(X_input)

    X1 = layers.Dropout(0.6, seed = 7)(A)
    X2 = layers.Dropout(0.6, seed = 7)(B)
    Y = layers.Dropout(0.6, seed = 7)(P)    
    
    def interaction(a, b):
        sub = layers.Lambda(lambda a: K.abs(a[0] - a[1]))([a, b])
        mult = layers.Lambda(lambda a: a[0] * a[1])([a, b])
        return layers.Concatenate()([a, b, sub, mult,])    
    
    word_encoder = layers.Dense(512, activation='selu')
    X1 = word_encoder(X1)
    X2 = word_encoder(X2)
    Y = word_encoder(Y)

    I_X1_Y = interaction(X1, Y)
    I_X2_Y = interaction(X2, Y)

    dense_encoder = layers.Dense(128, activation='selu')

    I_X1_Y = layers.Dropout(0.75)(dense_encoder(I_X1_Y))
    I_X2_Y = layers.Dropout(0.75)(dense_encoder(I_X2_Y))
    features = layers.Concatenate()([I_X1_Y, I_X2_Y])

    X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(features)
    X = layers.BatchNormalization(name = 'bn0')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed = 7)(X)

    # Output layer
    X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
    X = layers.Activation('softmax')(X)

    # Create model
    model = models.Model(input = X_input, output = X, name = "classif_model")
    return model

In [22]:
def make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_tag, model_func,
                dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
                oof_folder='oof/', pred_folder='test_outputs/'):

    tag = tag + str(layer) # follow the original naming style
    dev_filename = tag + dev_filename
    val_filename = tag + val_filename
    test_filename = tag + test_filename
    
    development = pd.read_json(os.path.join(dev_folder_path, dev_filename))
    X_development, Y_development = parse_json(development, embedding_size)

    # We want predictions for all development rows. So instead of removing rows, make them 0
    remove_development = [row for row in range(len(X_development)) if np.sum(np.isnan(X_development[row].reshape(-1)))]
    X_development[remove_development] = np.zeros((3 * embedding_size))

    prediction = np.zeros((len(X_development), 3)) # testing predictions

    for fold_n in range(n_fold):
        # split training and validation data
        print('Make predictions for', fold_n, 'started at', time.ctime())
        classif_model = model_func([X_development.shape[-1]], split_size=embedding_size)
        classif_model.load_weights(os.path.join(checkpoint_path, model_tag + tag + str(fold_n) + '.pt'))

        pred = classif_model.predict(x=X_development, verbose=0)
        prediction += pred
    
    prediction /= n_fold
    # prediction[remove_development] = [0.4, 0.4, 0.2] This is useless for prediction stage

    submission = pd.read_csv("data/sample_submission_stage_2.csv", index_col = "ID")
    submission["A"] = prediction[:,0]
    submission["B"] = prediction[:,1]
    submission["NEITHER"] = prediction[:,2]
    submission.to_csv(pred_folder + model_tag + tag + ".csv")

In [23]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-uncased-seq512-"
pred_tag = "nli-mh-"
model_func = build_mlp_model_for_base_bert

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

pred_tag = "bnli-mh-"
model_func = base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

Make predictions for 0 started at Fri Apr 19 21:53:46 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:53:49 2019
Make predictions for 2 started at Fri Apr 19 21:53:52 2019
Make predictions for 3 started at Fri Apr 19 21:53:56 2019
Make predictions for 4 started at Fri Apr 19 21:53:59 2019
Make predictions for 5 started at Fri Apr 19 21:54:03 2019
Make predictions for 6 started at Fri Apr 19 21:54:07 2019
Make predictions for 0 started at Fri Apr 19 21:54:14 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:54:16 2019
Make predictions for 2 started at Fri Apr 19 21:54:18 2019
Make predictions for 3 started at Fri Apr 19 21:54:20 2019
Make predictions for 4 started at Fri Apr 19 21:54:22 2019
Make predictions for 5 started at Fri Apr 19 21:54:24 2019
Make predictions for 6 started at Fri Apr 19 21:54:26 2019


In [24]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base_cased"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-cased-seq512-"
pred_tag = "nli-mh-"
model_func = build_mlp_model_for_base_bert

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

pred_tag = "bnli-mh-"
model_func = base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

Make predictions for 0 started at Fri Apr 19 21:54:32 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:54:36 2019
Make predictions for 2 started at Fri Apr 19 21:54:40 2019
Make predictions for 3 started at Fri Apr 19 21:54:43 2019
Make predictions for 4 started at Fri Apr 19 21:54:48 2019
Make predictions for 5 started at Fri Apr 19 21:54:52 2019
Make predictions for 6 started at Fri Apr 19 21:54:56 2019
Make predictions for 0 started at Fri Apr 19 21:55:03 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:55:06 2019
Make predictions for 2 started at Fri Apr 19 21:55:09 2019
Make predictions for 3 started at Fri Apr 19 21:55:12 2019
Make predictions for 4 started at Fri Apr 19 21:55:14 2019
Make predictions for 5 started at Fri Apr 19 21:55:18 2019
Make predictions for 6 started at Fri Apr 19 21:55:20 2019


In [25]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big"
embedding_size = 1024
layer = 19
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-uncased-seq300-"
pred_tag = "nli-mh-"
model_func = build_mlp_model_for_larget_bert

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

pred_tag = "bnli-mh-"
model_func = base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

Make predictions for 0 started at Fri Apr 19 21:55:28 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:55:32 2019
Make predictions for 2 started at Fri Apr 19 21:55:37 2019
Make predictions for 3 started at Fri Apr 19 21:55:42 2019
Make predictions for 4 started at Fri Apr 19 21:55:46 2019
Make predictions for 5 started at Fri Apr 19 21:55:51 2019
Make predictions for 6 started at Fri Apr 19 21:55:57 2019
Make predictions for 0 started at Fri Apr 19 21:56:06 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:56:10 2019
Make predictions for 2 started at Fri Apr 19 21:56:14 2019
Make predictions for 3 started at Fri Apr 19 21:56:17 2019
Make predictions for 4 started at Fri Apr 19 21:56:21 2019
Make predictions for 5 started at Fri Apr 19 21:56:24 2019
Make predictions for 6 started at Fri Apr 19 21:56:28 2019


In [26]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big_cased"
embedding_size = 1024
layer = 18
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-cased-seq300-"
pred_tag = "nli-mh-"
model_func = build_mlp_model_for_larget_bert

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

pred_tag = "bnli-mh-"
model_func = base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag, model_func=model_func)

Make predictions for 0 started at Fri Apr 19 21:56:37 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:56:42 2019
Make predictions for 2 started at Fri Apr 19 21:56:47 2019
Make predictions for 3 started at Fri Apr 19 21:56:53 2019
Make predictions for 4 started at Fri Apr 19 21:56:59 2019
Make predictions for 5 started at Fri Apr 19 21:57:05 2019
Make predictions for 6 started at Fri Apr 19 21:57:11 2019
Make predictions for 0 started at Fri Apr 19 21:57:21 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 21:57:26 2019
Make predictions for 2 started at Fri Apr 19 21:57:30 2019
Make predictions for 3 started at Fri Apr 19 21:57:35 2019
Make predictions for 4 started at Fri Apr 19 21:57:40 2019
Make predictions for 5 started at Fri Apr 19 21:57:44 2019
Make predictions for 6 started at Fri Apr 19 21:57:49 2019


## Predictons for stage 1.4

### Prepare Data

In [27]:
import spacy
import pandas as pd
import numpy as np
nlp = spacy.load('en_core_web_lg')

# Load the GAP data

data = pd.concat([pd.read_csv('fixed_test_stage_2.tsv', sep='\t'),
                  pd.read_csv('gap-validation.tsv', sep='\t'),
                  pd.read_csv('gap-test.tsv', sep='\t'), 
                 ], ignore_index = True)

In [29]:
def domain(t):
    while not t._.subj and not t._.poss and\
            not (t.dep_ == 'xcomp' and t.head._.obj) and\
            t != t.head:
        t = t.head
    return t

def ccom(t):
    return [t2 for t2 in t.head._.d]

# spacy extensions:
#   doc._.to(offset) => t at text char offset
#   t._.c => t's children (list)                     #   t._.d => t's descendents (list)
#   t._.subj => t's subject else False               #   t._.obj => t's object else False
#   t._.domain => t's syntactic domain               #   t._.ccom => t's c-command domain

spacy.tokens.doc.Doc.set_extension(
    'to', method=lambda doc, offset: [t for t in doc if t.idx == offset][0], force=True)
spacy.tokens.token.Token.set_extension(
    'c', getter=lambda t: [c for c in t.children], force=True)
spacy.tokens.token.Token.set_extension(
    'd', getter=lambda t: [c for c in t.sent if t in list(c.ancestors)], force=True)
spacy.tokens.token.Token.set_extension(
    'subj', getter=lambda t: ([c for c in t._.c if c.dep_.startswith('nsubj')] + [False])[0], force=True)
spacy.tokens.token.Token.set_extension(
    'obj', getter=lambda t: ([c for c in t._.c if c.dep_.startswith('dobj')] + [False])[0], force=True)
spacy.tokens.token.Token.set_extension(
    'poss', getter=lambda t: ([c for c in t._.c if c.dep_.startswith('poss')] + [False])[0], force=True)
spacy.tokens.token.Token.set_extension(
    'span', method=lambda t, t2: t.doc[t.i:t2.i] if t.i < t2.i else t.doc[t2.i:t.i], force=True)
spacy.tokens.token.Token.set_extension('domain', getter=domain, force=True)
spacy.tokens.token.Token.set_extension('ccom', getter=ccom, force=True)

def applyDisq(condition, candidates, candidate_dict, debug = False):
    badnames = sum([nameset(c, candidate_dict) for c in candidates if c in condition[0]], [])
    badcands = [c for c in candidates if c.text in badnames]
    if debug and len(badcands) > 0: print('Disqualified:', badcands, '<', condition[1])
    return [c for c in candidates if c not in badcands]

# Apply a list of disqualifying conditions
def applyDisqs(conditions, candidates, candidate_dict, debug = False):
    for condition in conditions:
        if len(candidates) < 1: return candidates
        candidates = applyDisq(condition, candidates, candidate_dict, debug)
    return candidates

# Pass the list of disqualifying conditions for possessive pronouns (his, her)
def disqGen(t, candidates, candidate_dict, debug = False):
    conds = [(t._.ccom,
             "disqualify candidates c-commanded by genpn; e.g. e.g. *Julia read his_i book about John_i's life."),
             ([t2 for t2 in candidates if t in t2._.ccom and t2.head.dep_ == 'appos'],
             "disqualify candidates modified by an appositive with genpn; e.g. *I wanted to see John_i, his_i father.")
            ]
    return applyDisqs(conds, candidates, candidate_dict, debug)

# Pass the list of list of disqualifying conditions for other pronouns
def disqOthers(t, candidates, candidate_dict, debug = False):
    conds = [([t2 for t2 in t._.ccom if t2.i > t.i],
             "disqualify candidates c-commanded by pn, unless they were preposed;\
             e.g. *He_i cried before John_i laughed. vs. Before John_i laughed, he_i cried."),
             ([t2 for t2 in candidates if t in t2._.ccom and t2._.domain == t._.domain
              and not (t.head.text == 'with' and t.head.head.lemma_ == 'take')],
             "disqualify candidates that c-command pn, unless in different domain;\
             e.g. Mary said that *John_i hit him_i. vs. John_i said that Mary hit him_i;\
             random hard-coded exception: `take with'"),
             ([t2 for t2 in candidates if t2._.domain.dep_ == 'xcomp' and t2._.domain.head._.obj and t2 == t2._.domain.head._.obj],
             "for xcomps with subjects parsed as upstairs dobj, disallow coref with that dobj;\
             e.g. *Mary wanted John_i to forgive him_i.")
            ]
    return applyDisqs(conds, candidates, candidate_dict, debug)

# Decide whether possessive or not and call appropriate function
def disq(t, candidates, candidate_dict, debug = False):
    func = disqGen if t.dep_ == 'poss' else disqOthers
    candidates = func(t, candidates, candidate_dict, debug)
    return candidates

def find_head(w, wo, doc):
    t = False; backtrack = 0
    while not t:
        try:
            t = doc._.to(wo)
        except IndexError:
            wo -= 1; backtrack += 1
    while t.dep_ == 'compound' and t.head.idx >= wo and t.head.idx < len(w) + wo + backtrack: t = t.head
    return t

# Returns subsequences of a name
def subnames(name):
    if type(name) != str: name = candidate_dict[name]
    parts = name.split(' ')
    subnames_ = []
    for i in range(len(parts)): 
        for j in range(i + 1, len(parts) + 1): 
            sub = ' '.join(parts[i:j])
            if len(sub) > 2: subnames_.append(sub)
    return subnames_

# Returns subsequences of a name unless potentially ambiguous (if another candidate picks out same subsequence)
def nameset(name, candidate_dict):
    if type(name) != str: name = candidate_dict[name]
    subnames_ = [sn for sn in subnames(name)]
    return [c for c in subnames_ if c not in sum([subnames(c) for c in candidate_dict.values() 
                                                  if c not in subnames_ and name not in subnames(c)], [])]

# Given the original candidate dict and the final candidate list, returns new dict grouping putative candidate instances under a single key
def candInstances(candidates, candidate_dict):
    candidates_by_name = {}
    for c in sorted(candidates, key = lambda c: len(candidate_dict[c]), reverse = True):
        name = candidate_dict[c]
        for name2 in candidates_by_name.keys():
            if name in nameset(name2, candidate_dict): name = name2; break
        candidates_by_name[name] = candidates_by_name.get(name, []) + [c]
    return candidates_by_name

import gender_guesser.detector as gender # oops
gd = gender.Detector()

# Needed to prune candidate dict-- removes non-provided candidates that don't match in most common gender with pn
def filterGender(candidates_by_name, a, b, pn):
    badnames = []
    gender = 'female' if pn in ['She', 'she', 'her', 'Her'] else 'male'
    for name in candidates_by_name.keys():
        if a in subnames(name) or b in subnames(name): continue
        genderii = gd.get_gender(name.split(' ')[0])
        if gender == 'male' and genderii == 'female': badnames += [name]; continue
        if gender == 'female' and genderii == 'male': badnames += [name]; continue
    for name in badnames: candidates_by_name.pop(name)
    return candidates_by_name

from urllib.parse import unquote
import re

# Authors' metric 1: Does the Wikipedia url contain the candidate's name?
def urlMatch(a, b, url, candidate_dict):
    url = re.sub('[^\x00-\x7F]', '*', unquote(url.split('/')[-1])).replace('_', ' ').lower()
    return {'a_url': (sorted([len(n.split(' ')) for n in nameset(a.lower(), candidate_dict) if n in nameset(url, candidate_dict)], reverse = True) + [0])[0],
            'b_url': (sorted([len(n.split(' ')) for n in nameset(b.lower(), candidate_dict) if n in nameset(url, candidate_dict)], reverse = True) + [0])[0]}

# Authors' metric 2: When pn is subject or object, does the candidate match?
def parallel(t1, t2):
    if t1.dep_.startswith('nsubj'): return t2.dep_.startswith('nsubj')
    if t1.dep_.startswith('dobj'): return t2.dep_.startswith('dobj')
    if t1.dep_.startswith('dative'): return t2.dep_.startswith('dative')
    return False

# Depth from a node to a parent node
def depthTo(t1, t2):
    depth = 0
    while t1 != t2 and t1 != t1.head:
        t1 = t1.head
        depth += 1
    return depth

# Syntactic distance within a single tree
def nodeDist(t1, t2):
    if t1 == t2: return 0
    if t2 in t1._.d: return depthTo(t2, t1)
    if t1 in t2._.d: return depthTo(t1, t2)
    t = t1
    while t1 not in t._.d or t2 not in t._.d and t != t.head: t = t.head
    return depthTo(t1, t) + depthTo(t2, t)

# Authors' metric 3: Syntactic distance (within or across trees)
def synDist(t, pn, doc, debug = False):
    doc_sents = list(doc.sents)
    sspan = doc_sents.index(pn.sent) - doc_sents.index(t.sent)
    if sspan == 0: # same sentence
        dist = nodeDist(t, pn)
    else: # different sentence
        dist = nodeDist(pn, doc_sents[doc_sents.index(pn.sent)].root) + nodeDist(t, doc_sents[doc_sents.index(t.sent)].root) # dist from two roots
    if debug: 
        print('pn dist:', nodeDist(pn, doc_sents[doc_sents.index(pn.sent)].root), '; t dist:',
              nodeDist(t, doc_sents[doc_sents.index(t.sent)].root), '; span:', sspan)
    sspan = abs(sspan) * 1 if sspan >= 0 else abs(sspan) * 1.3 # less local if not preceding
    return dist + sspan# * 0.7

# Character distance
def charDist(t1, t2):
    if t2.idx > t1.idx:
        return t2.idx - t1.idx + len(t1.text)
    else:
        return (t1.idx - t2.idx + len(t2.text)) * 1.3

# Theta prominence: assign a 0.1 to 1 score based on dep role of candidate -- strong feature
def thetaProminence(t, mult = 1, debug = False):
    while t.dep_ == 'compound': t = t.head
    if debug: print('t dep_:', t.dep_)
    if t.dep_ == 'pobj': mult = 1.3 if t.head.i < t.head.head.i else 1
    if t._.domain.dep_ == 'advcl': mult = 1.3 if t.head.i < t._.domain.head.i else 1
    if t.dep_.startswith('nsubj'): score = 1
    elif t.dep_.startswith('dobj'): score = 0.8
    elif t.dep_.startswith('dative'): score = 0.6
    elif t.dep_.startswith('pobj'): score = 0.4
    elif t.dep_.startswith('poss'): score = 0.3
    else: score = 0.1
    if debug: print('mult:', mult, '; score:', score)
    return min(1, score * mult)

# Computes these metrics for each candidate, and returns, for each group of instances (A instances, B instances,\
# other instances), either the sum, or the highest difference from the mean
def score(label, candidates_by_name, a_cand, b_cand, func, minsc = None, method = 'sum'):
    if method == 'sum':
        scores = {name: sum([func(t) for t in tokens]) for name, tokens in candidates_by_name.items()}
    elif method == 'meandiff':
        mean = np.mean(sum([[func(t) for t in tokens] for tokens in candidates_by_name.values()], []))
        scores = {name: mean - min([func(t) for t in tokens]) for name, tokens in candidates_by_name.items()}
    sca = scores[a_cand] if a_cand else minsc
    scb = scores[b_cand] if b_cand else minsc
    screst = [v for n, v in scores.items() if n != a_cand and n != b_cand]
    if method == 'sum':
        screst = sum(screst) if len(screst) > 0 else minsc
    elif method == 'meandiff':
        screst = max(screst) if len(screst) > 0 else minsc
    return {'a_' + label: sca, 'b_' + label: scb, 'n_' + label: screst}

from tqdm import tqdm_notebook as tqdm

# Load a rowfull of data
def load_row(data, i):
    return tuple(data.iloc[i])

# Row by row, populate features
def annotateSet(data, minsc = None, debug = False):
    
    annotated_data = pd.DataFrame() # init placeholder df
    row_batch = []

    for i in tqdm(range(annotated_data.shape[0], data.shape[0])):
        id, text, pn, pno, a, ao, ag, b, bo, bg, url = load_row(data, i)        
        doc = nlp(text) # parse text into doc
        try:
            pnt, at, bt = (doc._.to(pno), find_head(a, ao, doc), find_head(b, bo, doc)) # get the tokens that correspond to offsets
        except:
            # There is an error which has already been reported by 
            # https://www.kaggle.com/c/gendered-pronoun-resolution/discussion/89830#latest-518973
            # my fixing is simply replacing the `)` with a space ' ', so 'he' would match the offset and spacy also can work.
            doc = nlp(text.replace(')', ' '))
            pnt, at, bt = (doc._.to(pno), find_head(a, ao, doc), find_head(b, bo, doc))
            
        pnt, at, bt = (doc._.to(pno), find_head(a, ao, doc), find_head(b, bo, doc)) # get the tokens that correspond to offsets
        candidate_dict = {e.root: re.sub('\'s$', '', e.text) for e in [e for e in doc.ents if e.root.ent_type_ == 'PERSON']} # first get every PERSON ent as candidate
        candidate_dict.update({c.root: re.sub('\'s$', '', c.text) for c in doc.noun_chunks if c.root.pos_ == 'PROPN' and c.text in sum([subnames(n) for n in candidate_dict.values()], []) and
                               c.root not in candidate_dict.keys()}) # get some missed ones by looking at noun chunks with PROPN roots whose text match part of a candidate but are not already in list
        candidate_dict.update({t: w for t, w in [(at, a), (bt, b)]}) # add provided cands, overwriting in the process

        candidates = disq(pnt, list(candidate_dict.keys()), candidate_dict, debug = False)
        candidates_by_name = candInstances(candidates, candidate_dict)
        candidates_by_name = filterGender(candidates_by_name, a, b, pn)
        a_cand = ([name for name, tokens in candidates_by_name.items() if at in tokens] + [False])[0]
        b_cand = ([name for name, tokens in candidates_by_name.items() if bt in tokens] + [False])[0]
    
        # init row dict
        features = {'id': id, 'label': 0 if ag else 1 if bg else 2}
        # eliminated or not
        features.update({'a_out': 0 if a_cand else 1, 'b_out': 0 if b_cand else 1})
        # url match or not
        features.update(urlMatch(a, b, url, candidate_dict))
        # c-command or not
        features.update({'a_cc': 1 if a_cand and pnt in at._.ccom else 0, 'b_cc': 1 if b_cand and pnt in bt._.ccom else 0})
        # parallelism score
        features.update(score('par', candidates_by_name, a_cand, b_cand, lambda t: parallel(t, pnt), minsc = minsc))
        # theta prominence score
        features.update(score('th', candidates_by_name, a_cand, b_cand, thetaProminence, minsc = minsc))
        # syntactic distance score
        features.update(score('loc', candidates_by_name, a_cand, b_cand, lambda t: synDist(t, pnt, doc), method='meandiff', minsc = minsc))
        # number of candidates left
        features.update({'n_cands': len(candidates_by_name)})
        # char dist
        features.update(score('cloc', candidates_by_name, a_cand, b_cand, lambda t: charDist(t, pnt), method='meandiff', minsc = minsc))

        if debug: print(id, '>', 'a:', 1 if ag else 0, 'b:', 1 if bg else 0, features)
        row_batch += [features]

    # add rows to placeholder df
    if annotated_data.shape[0] != data.shape[0]: annotated_data = annotated_data.append(row_batch, ignore_index = True)
    
    return annotated_data

# Readable rows

from textwrap import TextWrapper
wrapper = TextWrapper(width=75)

def style(w, wstyle):
    if wstyle == None: return '{}{}{}'.format('\033[1m', w, '\033[0m') # bold
    elif wstyle == True: return '{}{}{}'.format('\033[92m', w, '\033[0m') # green
    elif wstyle == False: return '{}{}{}'.format('\033[91m', w, '\033[0m') # red

def readable_rows(data, range):
    rows = []
    for i in range:
        wois = sorted([(data.iloc[i]['Pronoun'], data.iloc[i]['Pronoun-offset'], None),\
                   (data.iloc[i]['A'], data.iloc[i]['A-offset'], data.iloc[i]['A-coref']),\
                   (data.iloc[i]['B'], data.iloc[i]['B-offset'], data.iloc[i]['B-coref'])],\
                   key = lambda x: x[1]) # sort by offset
        text = ''; ftext = data.iloc[i]['Text']; coffset = 0
        for w, woffset, wstyle in wois:
            text += ftext[coffset:woffset] + style(w, wstyle)
            coffset += len(ftext[coffset:woffset]) + len(w)
        text += ftext[coffset:]
        rows += [(str(i), data.iloc[i]['ID'], 'A' if data.iloc[i]['A-coref'] else 'B' if data.iloc[i]['B-coref'] else 'N', text)]
    return rows

def print_readable_rows(data, range):
    for index, id, target, text in readable_rows(data, range):
        text = '\n\t'.join(wrapper.wrap(text = text))
        print('{} ({}): {}>\n\t{}\n'.format(id, index, target, text))
        
def print_tokens(doc):
    for i in range(0, len(doc), 3):
        for t in doc[i:i+3]:
            print("{}[{}] >{}> {}".format(t.text, t.pos_, t.dep_, t.head.text), end = ' | ')
        print('')
        
annotated_data = annotateSet(data)

misdisqs_a = annotated_data.loc[(annotated_data['a_out'] == 1) & (annotated_data['label'] == 0)]
misdisqs_b = annotated_data.loc[(annotated_data['b_out'] == 1) & (annotated_data['label'] == 1)]

print('As wrongly disqualified:', misdisqs_a.shape[0], ', Bs wrongly disqualified:', misdisqs_b.shape[0], '\n')

for i in misdisqs_a.index[:1]: # the first
    print_readable_rows(data, [i])
    id, text, pn, pno, a, ao, ag, b, bo, bg, url = load_row(data,i)
    doc = nlp(text); print_tokens(doc)
    print('')
    
train = annotated_data.loc[(annotated_data['id'].str.contains('test'))] # swap later
valid = annotated_data.loc[(annotated_data['id'].str.contains('validation'))]
test = annotated_data.loc[(annotated_data['id'].str.contains('development'))] # swap later

# Note, the stage 2 test set should not be added into train_valid, or it might break the MinMaxScalar!!!
train_valid = pd.concat([train, valid, test])

answer_columns = ['label']
excl = ['id']
excl += ['a_out', 'b_out']
feature_columns = [col for col in annotated_data.columns if col not in answer_columns and col not in excl]

meta_features_train_valid = train_valid[feature_columns].fillna(0)
meta_features_test = test[feature_columns].fillna(0)

C:\Users\zake7\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\zake7\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


As wrongly disqualified: 36 , Bs wrongly disqualified: 44 

validation-12 (12370): A>
	Nicole flirts with Charlie and gives him her phone number. She later
	truants from school and visits Charlie. Nicole later arranges a date with
	Charlie, telling Michael she feels too ill to go to school. When
	Nicole kisses Charlie, Annie Sobacz (Reanne Farley) stops
	her and tells Charlie that Nicole is a school girl.

Nicole[PROPN] >nsubj> flirts | flirts[VERB] >ROOT> flirts | with[ADP] >prep> flirts | 
Charlie[PROPN] >pobj> with | and[CCONJ] >cc> flirts | gives[VERB] >conj> flirts | 
him[PRON] >dative> gives | her[ADJ] >poss> number | phone[NOUN] >compound> number | 
number[NOUN] >dobj> gives | .[PUNCT] >punct> flirts | She[PRON] >nsubj> truants | 
later[ADV] >advmod> truants | truants[NOUN] >ROOT> truants | from[ADP] >prep> truants | 
school[NOUN] >pobj> from | and[CCONJ] >cc> truants | visits[VERB] >conj> truants | 
Charlie[PROPN] >dobj> visits | .[PUNCT] >punct> truants | Nicole[PROPN] >nsubj>

In [30]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
for col in meta_features_train_valid.columns:
    std = MinMaxScaler()
    meta_features_train_valid[col] = std.fit_transform(meta_features_train_valid[col].values.reshape(-1, 1))
    meta_features_test[col] = std.transform(meta_features_test[col].values.reshape(-1, 1))
    
meta_features_test = meta_features_test.values
print("meta_features_test shpae:", meta_features_test.shape)    

C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


meta_features_test shpae: (12359, 17)


In [31]:
def build_mlp_model(input_shape, split_size, meta_features_shape):
    X_input = layers.Input(input_shape)
    meta_input = layers.Input(meta_features_shape)

    # First dense layer
    A = layers.Lambda(lambda x: x[:, :split_size])(X_input)
    B = layers.Lambda(lambda x: x[:, split_size:split_size*2])(X_input)
    P = layers.Lambda(lambda x: x[:, split_size*2:])(X_input)

    head_num = 6
    res = []
    for head in range(head_num):
        query_encoder = layers.Dense(dense_layer_sizes[0], activation='selu', kernel_regularizer = regularizers.l2(1e-6))
        ans_encoder = layers.Dense(dense_layer_sizes[0], activation='selu', kernel_regularizer = regularizers.l2(1e-6))
        
        d_ratio = 0.6
        
        a, b = query_encoder(layers.Dropout(d_ratio)(A)), query_encoder(layers.Dropout(d_ratio)(B))
        p = query_encoder(layers.Dropout(d_ratio)(P))
        
        amp = layers.Multiply()([a, p])
        bmp = layers.Multiply()([b, p])
        
        asp = layers.Lambda(lambda v: v[0] - v[1])([p, a])
        bsp = layers.Lambda(lambda v: v[0] - v[1])([p, b])        
        
        ia = layers.Concatenate()([a, p, amp, asp])
        ib = layers.Concatenate()([b, p, bmp, bsp])
        nli_encoder = layers.Dense(dense_layer_sizes[0], activation='selu')
        ia, ib = nli_encoder(ia), nli_encoder(ib)
        
        out = layers.Concatenate()([ia, ib])
        res.append(out)
    
    res = layers.Add()(res)
    res = layers.Dropout(0.8)(res)
    
    meta_out = layers.Dense(20, activation='selu')(meta_input)
    meta_out = layers.Dropout(0.25)(meta_out)    
    
    res = layers.Concatenate()([res, meta_out])
    
    X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(res)
    X = layers.BatchNormalization(name = 'bn0')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed = 7)(X)

    # Output layer
    X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
    X = layers.Activation('softmax')(X)

    # Create model
    model = models.Model(input = [X_input, meta_input], output = X, name = "classif_model")
    return model


def build_base_nli_model(input_shape, split_size, meta_features_shape):
    X_input = layers.Input(input_shape)
    meta_input = layers.Input(meta_features_shape)

    # First dense layer
    A = layers.Lambda(lambda x: x[:, :split_size])(X_input)
    B = layers.Lambda(lambda x: x[:, split_size:split_size*2])(X_input)
    P = layers.Lambda(lambda x: x[:, split_size*2:])(X_input)

    query_encoder = layers.Dense(512, activation='selu', kernel_regularizer = regularizers.l2(1e-6))
    ans_encoder = layers.Dense(512, activation='selu', kernel_regularizer = regularizers.l2(1e-6))

    d_ratio = 0.6

    a, b = query_encoder(layers.Dropout(d_ratio)(A)), query_encoder(layers.Dropout(d_ratio)(B))
    p = query_encoder(layers.Dropout(d_ratio)(P))

    amp = layers.Multiply()([a, p])
    bmp = layers.Multiply()([b, p])

    asp = layers.Lambda(lambda v: v[0] - v[1])([p, a])
    bsp = layers.Lambda(lambda v: v[0] - v[1])([p, b])        

    ia = layers.Concatenate()([a, p, amp, asp])
    ib = layers.Concatenate()([b, p, bmp, bsp])
    
    nli_encoder = layers.Dense(128, activation='selu')
    ia, ib = nli_encoder(ia), nli_encoder(ib)

    features = layers.Concatenate()([ia, ib])    
    features = layers.Dropout(0.75)(features)
    
    meta_out = layers.Dense(20, activation='selu')(meta_input)
    meta_out = layers.Dropout(0.25)(meta_out)    
    
    res = layers.Concatenate()([features, meta_out])
    
    X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(res)
    X = layers.BatchNormalization(name = 'bn0')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed = 7)(X)

    # Output layer
    X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
    X = layers.Activation('softmax')(X)

    # Create model
    model = models.Model(input = [X_input, meta_input], output = X, name = "classif_model")
    return model

In [32]:
def make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_tag, model_func, 
                dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
                oof_folder='oof/', pred_folder='outputs/'):

    tag = tag + str(layer) # follow the original naming style
    dev_filename = tag + dev_filename
    val_filename = tag + val_filename
    test_filename = tag + test_filename
    
    development = pd.read_json(os.path.join(dev_folder_path, dev_filename))
    X_development, Y_development = parse_json(development, embedding_size)

    remove_development = [row for row in range(len(X_development)) if np.sum(np.isnan(X_development[row].reshape(-1)))]
    X_development[remove_development] = np.zeros((3 * embedding_size))

    # Will predict probabilities for data from the gap-development file; initializing the predictions
    prediction = np.zeros((len(X_development), 3)) # testing predictions
    
    for fold_n in range(n_fold):
        # split training and validation data
        print('Make predictions for', fold_n, 'started at', time.ctime())
        # Define the model, re-initializing for each fold
        classif_model = model_func([X_development.shape[-1]], split_size=embedding_size, meta_features_shape=[meta_features_test.shape[-1]])
        classif_model.load_weights(os.path.join(checkpoint_path, model_tag + tag + str(fold_n) + '.pt'))

        pred = classif_model.predict(x=[X_development, meta_features_test], verbose=0)
        prediction += pred
    
    prediction /= n_fold
    
    submission = pd.read_csv("data/sample_submission_stage_2.csv", index_col = "ID")
    submission["A"] = prediction[:,0]
    submission["B"] = prediction[:,1]
    submission["NEITHER"] = prediction[:,2]
    submission.to_csv(pred_folder + model_tag + tag + ".csv")

In [33]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-uncased-seq512-"
pred_tag = "Hnli-mh-"
model_func = build_mlp_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

pred_tag = "HBnli-mh-"
model_func = build_base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:09:59 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:10:06 2019
Make predictions for 2 started at Fri Apr 19 22:10:12 2019
Make predictions for 3 started at Fri Apr 19 22:10:20 2019
Make predictions for 4 started at Fri Apr 19 22:10:27 2019
Make predictions for 5 started at Fri Apr 19 22:10:33 2019
Make predictions for 6 started at Fri Apr 19 22:10:40 2019
Make predictions for 0 started at Fri Apr 19 22:10:51 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:10:57 2019
Make predictions for 2 started at Fri Apr 19 22:11:02 2019
Make predictions for 3 started at Fri Apr 19 22:11:07 2019
Make predictions for 4 started at Fri Apr 19 22:11:13 2019
Make predictions for 5 started at Fri Apr 19 22:11:19 2019
Make predictions for 6 started at Fri Apr 19 22:11:24 2019


In [34]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base_cased"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-cased-seq512-"
pred_tag = "Hnli-mh-"
model_func = build_mlp_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

pred_tag = "HBnli-mh-"
model_func = build_base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:11:34 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:11:41 2019
Make predictions for 2 started at Fri Apr 19 22:11:49 2019
Make predictions for 3 started at Fri Apr 19 22:11:56 2019
Make predictions for 4 started at Fri Apr 19 22:12:05 2019
Make predictions for 5 started at Fri Apr 19 22:12:12 2019
Make predictions for 6 started at Fri Apr 19 22:12:20 2019
Make predictions for 0 started at Fri Apr 19 22:12:32 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:12:39 2019
Make predictions for 2 started at Fri Apr 19 22:12:45 2019
Make predictions for 3 started at Fri Apr 19 22:12:52 2019
Make predictions for 4 started at Fri Apr 19 22:12:58 2019
Make predictions for 5 started at Fri Apr 19 22:13:05 2019
Make predictions for 6 started at Fri Apr 19 22:13:12 2019


In [35]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big"
embedding_size = 1024
layer = 19
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-uncased-seq300-"
pred_tag = "Hnli-mh-"
model_func = build_mlp_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

pred_tag = "HBnli-mh-"
model_func = build_base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:13:23 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:13:32 2019
Make predictions for 2 started at Fri Apr 19 22:13:40 2019
Make predictions for 3 started at Fri Apr 19 22:13:49 2019
Make predictions for 4 started at Fri Apr 19 22:13:58 2019
Make predictions for 5 started at Fri Apr 19 22:14:07 2019
Make predictions for 6 started at Fri Apr 19 22:14:16 2019
Make predictions for 0 started at Fri Apr 19 22:14:30 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:14:38 2019
Make predictions for 2 started at Fri Apr 19 22:14:46 2019
Make predictions for 3 started at Fri Apr 19 22:14:54 2019
Make predictions for 4 started at Fri Apr 19 22:15:01 2019
Make predictions for 5 started at Fri Apr 19 22:15:09 2019
Make predictions for 6 started at Fri Apr 19 22:15:17 2019


In [36]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big_cased"
embedding_size = 1024
layer = 18
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-cased-seq300-"
pred_tag = "Hnli-mh-"
model_func = build_mlp_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

pred_tag = "HBnli-mh-"
model_func = build_base_nli_model

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_func=model_func,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:15:30 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:15:40 2019
Make predictions for 2 started at Fri Apr 19 22:15:50 2019
Make predictions for 3 started at Fri Apr 19 22:16:00 2019
Make predictions for 4 started at Fri Apr 19 22:16:10 2019
Make predictions for 5 started at Fri Apr 19 22:16:20 2019
Make predictions for 6 started at Fri Apr 19 22:16:30 2019
Make predictions for 0 started at Fri Apr 19 22:16:46 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:16:55 2019
Make predictions for 2 started at Fri Apr 19 22:17:03 2019
Make predictions for 3 started at Fri Apr 19 22:17:12 2019
Make predictions for 4 started at Fri Apr 19 22:17:21 2019
Make predictions for 5 started at Fri Apr 19 22:17:30 2019
Make predictions for 6 started at Fri Apr 19 22:17:39 2019


## Predictions for stage 1.5

In [37]:
def build_mlp_model(input_shape, split_size, meta_features_shape,):
    X_input = layers.Input(input_shape)
    
    meta_input = layers.Input(meta_features_shape)
    meta_out = layers.Dropout(0.1, seed = 7)(meta_input)
    meta_out = layers.Dense(20, activation='selu')(meta_out)
    
    X = layers.Dropout(dropout_rate, seed = 7)(X_input)
    X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(X)
    X = layers.Concatenate()([meta_out, X])
    
    X = layers.BatchNormalization(name = 'bn0')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed = 7)(X)

    # Output layer
    X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
    X = layers.Activation('softmax')(X)

    # Create model
    model = models.Model(input = [X_input, meta_input], output = X, name = "classif_model")
    return model

In [38]:
def make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_tag,
                dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
                oof_folder='oof/', pred_folder='outputs/'):

    tag = tag + str(layer) # follow the original naming style
    dev_filename = tag + dev_filename
    val_filename = tag + val_filename
    test_filename = tag + test_filename
    
    development = pd.read_json(os.path.join(dev_folder_path, dev_filename))
    X_development, Y_development = parse_json(development, embedding_size)

    remove_development = [row for row in range(len(X_development)) if np.sum(np.isnan(X_development[row].reshape(-1)))]
    X_development[remove_development] = np.zeros((3 * embedding_size))

    # Will predict probabilities for data from the gap-development file; initializing the predictions
    prediction = np.zeros((len(X_development), 3)) # testing predictions
    
    for fold_n in range(n_fold):
        # split training and validation data
        print('Make predictions for', fold_n, 'started at', time.ctime())
        # Define the model, re-initializing for each fold
        classif_model = build_mlp_model([X_development.shape[-1]], split_size=embedding_size, meta_features_shape=[meta_features_test.shape[-1]])
        classif_model.load_weights(os.path.join(checkpoint_path, model_tag + tag + str(fold_n) + '.pt'))

        pred = classif_model.predict(x=[X_development, meta_features_test], verbose=0)
        prediction += pred
    
    prediction /= n_fold
    
    submission = pd.read_csv("data/sample_submission_stage_2.csv", index_col = "ID")
    submission["A"] = prediction[:,0]
    submission["B"] = prediction[:,1]
    submission["NEITHER"] = prediction[:,2]
    submission.to_csv(pred_folder + model_tag + tag + ".csv")

In [39]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-uncased-seq512-"
pred_tag = "Hnaive-"

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:17:52 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:18:01 2019
Make predictions for 2 started at Fri Apr 19 22:18:09 2019
Make predictions for 3 started at Fri Apr 19 22:18:18 2019
Make predictions for 4 started at Fri Apr 19 22:18:26 2019
Make predictions for 5 started at Fri Apr 19 22:18:35 2019
Make predictions for 6 started at Fri Apr 19 22:18:43 2019


In [40]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base_cased"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-cased-seq512-"
pred_tag = "Hnaive-"

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:18:56 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:19:06 2019
Make predictions for 2 started at Fri Apr 19 22:19:15 2019
Make predictions for 3 started at Fri Apr 19 22:19:24 2019
Make predictions for 4 started at Fri Apr 19 22:19:33 2019
Make predictions for 5 started at Fri Apr 19 22:19:42 2019
Make predictions for 6 started at Fri Apr 19 22:19:51 2019


In [41]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big"
embedding_size = 1024
layer = 19
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-uncased-seq300-"
pred_tag = "Hnaive-"

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:20:05 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:20:14 2019
Make predictions for 2 started at Fri Apr 19 22:20:23 2019
Make predictions for 3 started at Fri Apr 19 22:20:33 2019
Make predictions for 4 started at Fri Apr 19 22:20:42 2019
Make predictions for 5 started at Fri Apr 19 22:20:51 2019
Make predictions for 6 started at Fri Apr 19 22:21:00 2019


In [42]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big_cased"
embedding_size = 1024
layer = 18
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-cased-seq300-"
pred_tag = "Hnaive-"

make_predictions(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='test_outputs/', model_tag=pred_tag)

Make predictions for 0 started at Fri Apr 19 22:21:15 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


Make predictions for 1 started at Fri Apr 19 22:21:25 2019
Make predictions for 2 started at Fri Apr 19 22:21:34 2019
Make predictions for 3 started at Fri Apr 19 22:21:43 2019
Make predictions for 4 started at Fri Apr 19 22:21:52 2019
Make predictions for 5 started at Fri Apr 19 22:22:01 2019
Make predictions for 6 started at Fri Apr 19 22:22:10 2019


## Predictons for stage 1.42

## Load the predictions of base models

In [43]:
os.listdir('test_outputs')

['bnli-mh-bert-base-cased-seq512-8.csv',
 'bnli-mh-bert-base-uncased-seq512-8.csv',
 'bnli-mh-bert-large-cased-seq300-18.csv',
 'bnli-mh-bert-large-uncased-seq300-19.csv',
 'HBnli-mh-bert-base-cased-seq512-8.csv',
 'HBnli-mh-bert-base-uncased-seq512-8.csv',
 'HBnli-mh-bert-large-cased-seq300-18.csv',
 'HBnli-mh-bert-large-uncased-seq300-19.csv',
 'Hnaive-bert-base-cased-seq512-8.csv',
 'Hnaive-bert-base-uncased-seq512-8.csv',
 'Hnaive-bert-large-cased-seq300-18.csv',
 'Hnaive-bert-large-uncased-seq300-19.csv',
 'Hnli-mh-bert-base-cased-seq512-8.csv',
 'Hnli-mh-bert-base-uncased-seq512-8.csv',
 'Hnli-mh-bert-large-cased-seq300-18.csv',
 'Hnli-mh-bert-large-uncased-seq300-19.csv',
 'naive-bert-base-cased-seq512-8.csv',
 'naive-bert-base-uncased-seq512-8.csv',
 'naive-bert-large-cased-seq300-18.csv',
 'naive-bert-large-uncased-seq300-19.csv',
 'nli-mh-bert-base-cased-seq512-8.csv',
 'nli-mh-bert-base-uncased-seq512-8.csv',
 'nli-mh-bert-large-cased-seq300-18.csv',
 'nli-mh-bert-large-unca

In [45]:
preds = [pd.read_csv(os.path.join('test_outputs', pred_file)).values[:, 1:] for pred_file in pred_files]
test_x = np.concatenate(preds, axis=1)
print("preds shape", test_x.shape)

preds shape (12359, 72)


In [46]:
import keras
from keras.models import Model
from keras.layers import *
from sklearn.model_selection import KFold
from keras.optimizers import *

In [47]:
def build_mlp_model(feature_nums):
    features_inputs = Input(shape=feature_nums, name='mata-features', dtype="float32")
    features = features_inputs
    
    depth = 5
    for i in range(depth):
        new_features = Dense(24, activation='relu')(features)
        new_features = Dropout(0.5)(new_features)
        features = Concatenate()([features, new_features])

    out_ = Dense(3, activation='softmax')(features)
    
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy',])
    model.summary()
    return model

In [48]:
prediction = np.zeros((len(test_x), 3)) # testing predictions
n_fold = 7
for fold_n in range(n_fold):
    classif_model = build_mlp_model([test_x.shape[-1]])
    classif_model.load_weights('stage_1_finals/nn_ensemble_checkpoint_f{}.pt'.format(fold_n))
    pred = classif_model.predict(x=test_x, verbose=0)
    prediction += pred
    
prediction /= n_fold

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 72)           0                                            
__________________________________________________________________________________________________
dense_1233 (Dense)              (None, 24)           1752        mata-features[0][0]              
__________________________________________________________________________________________________
dropout_1657 (Dropout)          (None, 24)           0           dense_1233[0][0]                 
__________________________________________________________________________________________________
concatenate_1275 (Concatenate)  (None, 96)           0           mata-features[0][0]              
                                                                 dropout_1657[0][0]               
__________

__________________________________________________________________________________________________
dropout_1676 (Dropout)          (None, 24)           0           dense_1255[0][0]                 
__________________________________________________________________________________________________
concatenate_1294 (Concatenate)  (None, 192)          0           concatenate_1293[0][0]           
                                                                 dropout_1676[0][0]               
__________________________________________________________________________________________________
dense_1256 (Dense)              (None, 3)            579         concatenate_1294[0][0]           
Total params: 15,099
Trainable params: 15,099
Non-trainable params: 0
__________________________________________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output 

In [49]:
# make a submission
submission = pd.read_csv("data/sample_submission_stage_2.csv", index_col = "ID")
submission["A"] = prediction[:,0]
submission["B"] = prediction[:,1]
submission["NEITHER"] = prediction[:,2]
submission.to_csv("stage_2_finals/s3_24_nn_ensembles.csv")    